# 如何缓存聊天模型响应
:::info 前提条件
本指南假设您熟悉以下概念：- [聊天模型](/docs/concepts/chat_models)- [大型语言模型（LLMs）](/docs/concepts/text_llms)
:::
LangChain 为[聊天模型](/docs/concepts/chat_models)提供了可选的缓存层。这一功能主要基于两个重要原因：
- 如果你经常多次请求相同的补全内容，它可以通过减少向LLM提供商发起的API调用次数来节省成本。这在应用程序开发期间尤为实用。- 它可以通过减少对LLM提供商的API调用次数来加速您的应用程序。
本指南将引导您了解如何在您的应用程序中启用此功能。

import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs customVarName="llm" />


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI()

In [2]:
# <!-- ruff: noqa: F821 -->
from langchain_core.globals import set_llm_cache

## 内存缓存
这是一个临时缓存，用于在内存中存储模型调用。当您的环境重启时，该缓存将被清空，并且不会在进程间共享。

In [3]:
%%time
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 645 ms, sys: 214 ms, total: 859 ms
Wall time: 829 ms


AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6836bdd-8c30-436b-828f-0ac5fc9ab50e-0')

In [4]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 822 µs, sys: 288 µs, total: 1.11 ms
Wall time: 1.06 ms


AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6836bdd-8c30-436b-828f-0ac5fc9ab50e-0')

## SQLite 缓存
此缓存实现使用 `SQLite` 数据库存储响应数据，并支持进程重启后持久化保存。

In [5]:
!rm .langchain.db

In [6]:
# We can do the same thing with a SQLite cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [7]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 9.91 ms, sys: 7.68 ms, total: 17.6 ms
Wall time: 657 ms


AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field!', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-39d9e1e8-7766-4970-b1d8-f50213fd94c5-0')

In [8]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 52.2 ms, sys: 60.5 ms, total: 113 ms
Wall time: 127 ms


AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field!', id='run-39d9e1e8-7766-4970-b1d8-f50213fd94c5-0')

## 后续步骤
你现在已经学会了如何缓存模型响应以节省时间和金钱。
接下来，请查阅本节中的其他聊天模型操作指南，例如[如何让模型返回结构化输出](/docs/how_to/structured_output)或[如何创建自定义聊天模型](/docs/how_to/custom_chat_model)。